<a href="https://colab.research.google.com/github/Emma922/Marketing-E-Commerce-Analysis/blob/main/Marketing_%26_E_Commerce_EDA_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marketing & E-Commerce EDA and Cleaning

## 1. Introduction  
This project performs a complete **Exploratory Data Analysis (EDA)**, and **Data Cleaning**,

The goal is to explore the dataset’s structure, relationships, and distributions, assess data quality issues like missing values or outliers, and then clean and transform the data for consistency. This ensures the dataset is accurate and business‑ready before moving into funnel, campaign, A/B test, and customer/product analysis.

We will:

- Load and explore all dataset tables (customers, products, campaigns, events, transactions)  
- Assess data structure, quality, and relationships across tables  
- Perform cleaning, detect outliers, and export tables.


# 2. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")
sns.set_theme()


# 3. Load the Datasets

Below is a description of each file:

| File         | Description                                                                 |
|:------------ |:----------------------------------------------------------------------------|
| customers    | Customer details (country, age, gender, and signup_date for segmentation and cohort analysis). |
| products     | Catalog of products with category, brand, base price, launch timing, and premium flag. |
| campaigns    | Marketing campaign details (channel, objective, dates, target segments). |
| events       | Fact table of user interactions (views, clicks, carts, purchases). Includes campaign_id and experiment_group for uplift and A/B testing. |
| transactions | One row per completed purchase, including quantity, discounts, and gross revenue (refunds as negatives). |








In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
events = pd.read_csv("events.csv")
products = pd.read_csv("products (2).csv")
transactions = pd.read_csv("transactions.csv")
customers = pd.read_csv("customers (2).csv")
campaigns = pd.read_csv("campaigns.csv")

# 4. Preview & Raw Exploration

We begin by inspecting the shape, structure, head, and summary of each dataset.


In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}

for name, df in dfs.items():
    print(f"======== {name.upper()} ========")

    display(df.head(3))

    print("\nInfo:")
    display(df.info())

    print("\nDescribe:")
    display(df.describe(include='all')
)

    print("\n\n")



# 5. Data Cleaning & Pre-processing

## 5.1 Missing values overview

In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}
for name, df in dfs.items():
    print(f"\nNull count for {name}:")
    print(df.isnull().sum())


## 5.2 Missing Numeric Values
- We'll look whether the dataset have missing values

In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}

for name, df in dfs.items():
    print(f"===== {name.upper()} =====")

    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_numeric = df[numeric_cols].isna().sum()

    print("Numeric columns with missing values:")
    print(missing_numeric[missing_numeric > 0])

    print("\n")


## 5.3 Outliers
- An outlier analysis will be performed to identify unusually extreme values that could distort the results of the study.


In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}

for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    corr = numeric_df.corr()

    if corr.empty:
        print(f"\nSkipping {name} because correlation matrix is empty or no numeric data.")
        continue

    # Plot boxplot for numeric columns to detect outliers
    plt.figure(figsize=(12, 6))
    numeric_df.boxplot()
    plt.title(f'Boxplot for Numeric Columns of {name}')
    plt.xticks(rotation=75)
    plt.show()


## 5.4 Duplicates Overview

In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}

for name, df in dfs.items():
    print(f"\nDuplicate rows for {name}:")
    duplicates = df[df.duplicated()]
    if duplicates.empty:
        print("No duplicate rows found.")
    else:
        print(duplicates)


## 5.4.1 Handling Duplicates

- No duplicate records were identified in the majority the tables

## 5.5 Handling Missing values

### Strategy:
- product_id: Cannot be imputed because is a key identifier; rows with missing product_id are excluded from product-level analysis.
- gross_revenue: Missing values are not imputed because they still provide useful context for customer, campaign, and discount analysis.
- device_type: Missing values are replaced by 'unknown'

- Categorical columns: Missing values are replaced with "Unknown" to maintain categorical integrity and avoid incorrect assumptions.

- Numeric columns: Rare missing values are imputed with the median; if missingness is widespread and the column is not essential, the feature may be removed.

- High-missingness irrelevant columns: Columns containing a high proportion of missing data and not neccesary to the analysis will be dropped



## 5.5.1 Replace missing values
- Missing values are replaced by 'unknown' in device type


In [ ]:
# Replace missing values in device_type with 'Unknown'
events['device_type'] = events['device_type'].fillna('Unknown')



## 5.5.2 Replace outliers with avg values

- The boxplot analysis shows no presence of significant outliers across the numeric variables.

- All detected values fall within expected and reasonable ranges for the dataset.

- Because of this, no outlier treatment is required

# 6. Correlations
- The goal of this step is to identify the relationships between the variables within the dataset.

- Understanding these correlations helps guide a more targeted and meaningful analysis, revealing how different features interact and potentially influence one another.

- A correlation heatmap will be used to visually examine these relationships, allowing for a clearer interpretation of both the strength and direction of the associations.

In [ ]:
dfs = {
    "customers": customers,
    "events": events,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns
}


for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    print(f"\nCorrelation matrix for {name}:")
    print(numeric_df.corr())



# 6.1 Heatmap




In [ ]:
for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    corr = numeric_df.corr()

    if corr.empty:
        print(f"\nSkipping {name} because correlation matrix is empty or no numeric data.")
        continue

    plt.figure(figsize=(18,4))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
    plt.title(f'Correlation Matrix Heatmap for {name}')
    plt.show()



## 6.2 Heatmap Insights

The dataset does not reveal any surprising correlations, with most variables operating independently. The only expected relationship is between base_price and is_premium, reflecting the logic that higher-priced products align with premium quality. Additionally, a moderate correlation (0.49) is observed between gross_revenue and quantity, consistent with larger purchases generating more revenue. Beyond these, no other meaningful correlations are present.



# 7. Summary


- Data Quality: The datasets overall exhibit good quality with manageable missing data.

- Missing Data: While some missing values exist, particularly in product_id, these were preserved intentionally to avoid introducing bias through imputation or deletion into the other variables.

- Outliers: Exploratory visualizations such as boxplots revealed no significant outliers that could potentially distort analyses or modeling outcomes, indicating the dataset's consistency.

- Duplicates: There were no duplicates in the datasets, which represents a adequate data integraty

- Correlations: Correlation heatmaps highlighted expected relationships among numeric variables

- **Next Steps:** The prepared datasets are now ready for advanced SQL analysis, and Tableau dashboard,
This structured cleaning and exploration provide a solid foundation for accurate funnel evaluation, campaign impact measurement, A/B test outcomes, and customer and product performance insights.


# 8 Export data to csv

In [ ]:
events.to_csv('events.csv', index=False)
products.to_csv('products.csv', index=False)
transactions.to_csv('transactions.csv', index=False)
customers.to_csv('customers.csv', index=False)
campaigns.to_csv('campaigns.csv', index=False)



In [ ]:
from google.colab import files

files.download('events.csv')
files.download('products.csv')
files.download('transactions.csv')
files.download('customers.csv')
files.download('campaigns.csv')

